# 参数初始化
参数初始化对模型具有较大的影响，不同的初始化方式可能会导致截然不同的结果，所幸的是很多深度学习的先驱们已经帮我们探索了各种各样的初始化方式，所以我们只需要学会如何对模型的参数进行初始化的赋值即可。

PyTorch 的初始化方式并没有那么显然，如果你使用最原始的方式创建模型，那么需要定义模型中的所有参数，当然这样可以非常方便地定义每个变量的初始化方式。但是对于复杂的模型，这并不容易，而且推荐使用 Sequential 和 Module 来定义模型，所以这个时候就需要知道如何来自定义初始化方式。

## 1. 使用 NumPy 来初始化
因为 PyTorch 是一个非常灵活的框架，理论上能够对所有的 Tensor 进行操作，所以我们能够通过定义新的 Tensor 来初始化，直接看下面的例子

In [1]:
import numpy as np
import torch
from torch import nn

In [2]:
# 定义一个 Sequential 模型
net1 = nn.Sequential(
    nn.Linear(30, 40),
    nn.ReLU(),
    nn.Linear(40, 50),
    nn.ReLU(),
    nn.Linear(50, 10)
)

In [3]:
# 访问第一层的参数
w1 = net1[0].weight
b1 = net1[0].bias

In [4]:
print(w1)

Parameter containing:
tensor([[-0.1169,  0.1221, -0.0782,  ..., -0.1810,  0.0432,  0.0630],
        [ 0.1002, -0.1319, -0.0284,  ..., -0.1502,  0.0087,  0.0961],
        [-0.1436,  0.1232, -0.1444,  ..., -0.0208,  0.0047, -0.0838],
        ...,
        [ 0.0831,  0.0809,  0.0504,  ..., -0.0222, -0.0338, -0.0483],
        [-0.1219, -0.1753,  0.1515,  ...,  0.0694,  0.1679, -0.0183],
        [ 0.0606, -0.0573, -0.1699,  ...,  0.0282,  0.0973, -0.0292]],
       requires_grad=True)


注意，这是一个 Parameter，也就是一个特殊的 Variable，我们可以访问其 `.data`属性得到其中的数据，然后直接定义一个新的 Tensor 对其进行替换，我们可以使用 PyTorch 中的一些随机数据生成的方式，比如 `torch.randn`，如果要使用更多 PyTorch 中没有的随机化方式，可以使用 numpy

In [6]:
# 定义一个 Tensor 直接对其进行替换
net1[0].weight.data = torch.from_numpy(np.random.uniform(3, 5, size=(40, 30)))

In [7]:
print(net1[0].weight)

Parameter containing:
tensor([[4.0991, 3.8642, 4.8695,  ..., 4.8893, 4.2825, 4.2195],
        [3.7516, 4.4228, 3.8516,  ..., 3.2370, 3.2288, 4.7692],
        [3.4961, 3.7355, 4.3391,  ..., 4.6747, 3.1574, 3.8553],
        ...,
        [3.1541, 3.7801, 4.5609,  ..., 4.9613, 3.1880, 4.7136],
        [3.6912, 4.0421, 4.1661,  ..., 3.6094, 3.9627, 4.6674],
        [4.6995, 3.8459, 3.5619,  ..., 3.6508, 4.1169, 3.8557]],
       dtype=torch.float64, requires_grad=True)


可以看到这个参数的值已经被改变了，也就是说已经被定义成了我们需要的初始化方式，如果模型中某一层需要我们手动去修改，那么我们可以直接用这种方式去访问，但是更多的时候是模型中相同类型的层都需要初始化成相同的方式，这个时候一种更高效的方式是使用循环去访问，比如

In [8]:
for layer in net1:
    if isinstance(layer, nn.Linear): # 判断是否是线性层
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 
        # 定义为均值为 0，方差为 0.5 的正态分布

一种非常流行的初始化方式叫 Xavier，方法来源于 2010 年的一篇论文 [Understanding the difficulty of training deep feedforward neural networks](http://proceedings.mlr.press/v9/glorot10a.html)，其通过数学的推导，证明了这种初始化方式可以使得每一层的输出方差是尽可能相等。这种初始化的公式为：

$$
w\ \sim \ Uniform[- \frac{\sqrt{6}}{\sqrt{n_j + n_{j+1}}}, \frac{\sqrt{6}}{\sqrt{n_j + n_{j+1}}}]
$$

其中 $n_j$ 和 $n_{j+1}$ 表示该层的输入和输出数目，所以请尝试实现以下这种初始化方式

对于 Module 的参数初始化，其实也非常简单，如果想对其中的某层进行初始化，可以直接像 Sequential 一样对其 Tensor 进行重新定义，其唯一不同的地方在于，如果要用循环的方式访问，需要介绍两个属性，children 和 modules，下面我们举例来说明

In [10]:
class Simp_Net(nn.Module):
    def __init__(self):
        super(Simp_Net, self).__init__()
        self.l1 = nn.Sequential(
            nn.Linear(30, 40),
            nn.ReLU()
        )
        
        self.l1[0].weight.data = torch.randn(40, 30) # 直接对某一层初始化
        
        self.l2 = nn.Sequential(
            nn.Linear(40, 50),
            nn.ReLU()
        )
        
        self.l3 = nn.Sequential(
            nn.Linear(50, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        return x

In [11]:
net2 = Simp_Net()

In [12]:
# 访问 children
for i in net2.children():
    print(i)

Sequential(
  (0): Linear(in_features=30, out_features=40, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=40, out_features=50, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=50, out_features=10, bias=True)
  (1): ReLU()
)


In [13]:
# 访问 modules
for i in net2.modules():
    print(i)

Simp_Net(
  (l1): Sequential(
    (0): Linear(in_features=30, out_features=40, bias=True)
    (1): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=40, out_features=50, bias=True)
    (1): ReLU()
  )
  (l3): Sequential(
    (0): Linear(in_features=50, out_features=10, bias=True)
    (1): ReLU()
  )
)
Sequential(
  (0): Linear(in_features=30, out_features=40, bias=True)
  (1): ReLU()
)
Linear(in_features=30, out_features=40, bias=True)
ReLU()
Sequential(
  (0): Linear(in_features=40, out_features=50, bias=True)
  (1): ReLU()
)
Linear(in_features=40, out_features=50, bias=True)
ReLU()
Sequential(
  (0): Linear(in_features=50, out_features=10, bias=True)
  (1): ReLU()
)
Linear(in_features=50, out_features=10, bias=True)
ReLU()


通过上面的例子，看到区别了吗?

children 只会访问到模型定义中的第一层，因为上面的模型中定义了三个 Sequential，所以只会访问到三个 Sequential，而 modules 会访问到最后的结构，比如上面的例子，modules 不仅访问到了 Sequential，也访问到了 Sequential 里面，这就对我们做初始化非常方便，比如

In [12]:
for layer in net2.modules():
    if isinstance(layer, nn.Linear):
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 

这上面实现了和 Sequential 相同的初始化，同样非常简便

## 2. `torch.nn.init`
因为 PyTorch 灵活的特性，可以直接对 Tensor 进行操作从而初始化，PyTorch 也提供了初始化的函数帮助我们快速初始化，就是 `torch.nn.init`，其操作层面仍然在 Tensor 上，下面我们举例说明

In [14]:
from torch.nn import init

In [15]:
print(net1[0].weight)

Parameter containing:
tensor([[-0.3018, -0.2876,  0.2165,  ..., -0.2142, -0.3176,  0.5495],
        [ 0.0388,  0.3076,  0.0590,  ..., -0.6364, -0.0267,  0.1603],
        [-0.2561, -0.2920, -0.6583,  ..., -0.8427,  0.3347,  0.7409],
        ...,
        [ 0.2859,  0.4651,  0.6217,  ...,  0.5055, -0.2478,  0.3720],
        [ 0.2158,  0.4416, -1.0657,  ..., -0.5606,  0.5312, -0.1419],
        [-0.0493, -0.6189, -0.4762,  ...,  0.3781, -0.3133,  0.2380]],
       dtype=torch.float64, requires_grad=True)


In [16]:
init.xavier_uniform_(net1[0].weight) # 这就是上面我们讲过的 Xavier 初始化方法，PyTorch 直接内置了其实现

Parameter containing:
tensor([[-0.1889,  0.1261,  0.2514,  ..., -0.1288, -0.2304, -0.2025],
        [ 0.1386,  0.1783, -0.1570,  ...,  0.2174,  0.1235,  0.0915],
        [ 0.0289,  0.0149,  0.2512,  ..., -0.2464, -0.1072,  0.0025],
        ...,
        [-0.2675,  0.0128, -0.1033,  ...,  0.2150,  0.0549,  0.1088],
        [ 0.0036, -0.2434, -0.0817,  ...,  0.2505, -0.2760, -0.1379],
        [-0.0121,  0.2645, -0.0503,  ..., -0.1788, -0.0196, -0.2303]],
       dtype=torch.float64, requires_grad=True)

In [17]:
print(net1[0].weight)

Parameter containing:
tensor([[-0.1889,  0.1261,  0.2514,  ..., -0.1288, -0.2304, -0.2025],
        [ 0.1386,  0.1783, -0.1570,  ...,  0.2174,  0.1235,  0.0915],
        [ 0.0289,  0.0149,  0.2512,  ..., -0.2464, -0.1072,  0.0025],
        ...,
        [-0.2675,  0.0128, -0.1033,  ...,  0.2150,  0.0549,  0.1088],
        [ 0.0036, -0.2434, -0.0817,  ...,  0.2505, -0.2760, -0.1379],
        [-0.0121,  0.2645, -0.0503,  ..., -0.1788, -0.0196, -0.2303]],
       dtype=torch.float64, requires_grad=True)


可以看到参数已经被修改了

`torch.nn.init` 提供了更多的内置初始化方式，避免了重复去实现一些相同的操作。

上面讲了两种初始化方式，其实它们的本质都是一样的，就是去修改某一层参数的实际值，而 `torch.nn.init` 提供了更多成熟的深度学习相关的初始化方式。
